In [1]:
import numpy as np
from numba import njit
from matplotlib import pyplot as plt
import plotly.graph_objects as go

In [112]:
error_rate_bound_list = np.array((.535, .525, .515, .510, .5, .48, .3))
conf = {'num_trials' : int(1e3),
        'num_steps' : int(1e4),
        'error_probability' : .5,
        'error_rate_bound_list' : error_rate_bound_list}

In [113]:
@njit
def _simulate(trials, num_trials, num_steps, error_rate_bound_list):
    signals = np.zeros((len(error_rate_bound_list), num_trials, num_steps))
    for i in range(len(error_rate_bound_list)):
        for k in range(num_steps):
            error_bound = error_rate_bound_list[i]*(k+1)
            for j in range(num_trials):
                signals[i,j,k] = trials[j,k]>error_bound
    return signals

def simulate(num_trials, num_steps, error_probability, error_rate_bound_list):
    trials = np.random.binomial(1, error_probability, (num_trials, num_steps))
    trials = np.cumsum(trials, axis = 1)
    return _simulate(trials = trials, num_trials=num_trials, num_steps=num_steps, error_rate_bound_list=error_rate_bound_list)
    

In [114]:
signals = simulate(**conf)
expected_signals = signals.mean(axis=1)
x_axis = np.arange(conf['num_steps'])
pdf_signals = expected_signals / expected_signals.sum(axis=1)[:,np.newaxis]
cumulative_signals = expected_signals.cumsum(axis=1)/expected_signals.sum(axis=1)[:,np.newaxis]


In [54]:
simulate(**conf)
%timeit simulate(**conf)

KeyboardInterrupt: 

In [116]:
fig = go.Figure(
    layout=go.Layout(
        title=f"Probability Density Function for Choices of error_rate_bound when the probability of error equals {conf['error_probability']}"
    ))
for i in range(len(error_rate_bound_list)):
    fig.add_trace(
    go.Scatter(x=x_axis[-1000:],
               y=pdf_signals[i][-1000:],
               name=f"Error Rate Bound of {error_rate_bound_list[i]}",
               visible=True))
fig